In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Assuming 'cleanData.py' has been provided and correctly cleans the data
from cleanData import clean

# Read the dataset
train_data = pd.read_csv('train.csv', low_memory=False)

# Clean the data using the 'clean' function from the 'cleanData.py' module
clean_data = clean(train_data)

# Separate features and target
X = clean_data.drop('Score', axis=1)
y = clean_data['Score']

# Check for any data leakage
for column in X.columns:
    if X[column].nunique() == y.nunique() and all(X[column].unique() == y.unique()):
        print(f"Possible data leakage detected in column: {column}")

# Define a function to create a model (required for KerasClassifier)
def create_model():
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train_transformed.shape[1],)),
        Dense(64, activation='relu'),
        Dense(y_encoded.shape[1], activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(exclude=['int64', 'float64']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Create the KerasClassifier wrapper
keras_model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=128, verbose=0)

# Create the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', keras_model)])

# Perform cross-validation
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5)

print("Cross-validation scores:", cv_scores)
print("Mean cross-validation score:", cv_scores.mean())

# If cross-validation scores are consistently high, it might not be overfitting or data leakage
# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Predict on the test data
y_pred = pipeline.predict(X_test)

# Calculate the accuracy on the test data
test_accuracy = accuracy_score(y_test, y_pred)

print(f"Test Accuracy: {test_accuracy}")


ModuleNotFoundError: No module named 'tensorflow.keras.wrappers'